Import packages and setting parameters

In [1]:
# packages
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import os, logging, sys, glob
from astropy import units as u
from lstchain.reco.utils import get_effective_time
pd.set_option("display.max_columns", None)

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# --- logging --- #
import logging
logger = logging.getLogger(__name__)
logger.addHandler(logging.StreamHandler())
logger.setLevel(logging.INFO)

from magicctapipe.io import get_dl2_mean

# location of the scripts
sys.path.insert(0, '/fefs/aswg/workspace/juan.jimenez/stereo_analysis/scripts')
import auxiliar as aux
import geometry as geom
aux.params()

# --- parameters --- #
# name of the studied source
source_name = 'Mrk421'
# simple, variance or intesity
weight_type = 'variance'
# ------------------ #

# --- file paths --- #
output_path = '/fefs/aswg//workspace/juan.jimenez/data/dl2/mc'
# ------------------ #

In [2]:
# selecting the MC folder
if source_name == 'Crab':
    path_mcs = '/fefs/aswg/LST1MAGIC/mc/DL2/ST0316A/NSB1.5/GammaTest/v01.2/dec_2276/dl2_gamma_zd_4*.h5' # for CrabNebula
elif source_name == 'BLLac':
    path_mcs = '/fefs/aswg/LST1MAGIC/mc/DL2/ST0316A/NSB0.5/GammaTest/v01.2/dec_3476/dl2_gamma_zd_4*.h5' # for BLLac
elif source_name == 'Mrk421':
    path_mcs = '/fefs/aswg/LST1MAGIC/mc/DL2/ST0316A/NSB0.5/GammaTest/v01.2/dec_4822/dl2_gamma_zd_4*.h5' # for Mrk421
else:
    logger.error(f'The source introduced {source_name} have not specified RFs or is bad written.')
    
logger.info(f'Study of the source: {source_name}')
logger.info(f'\nAll data taken from the path: {path_mcs}')

Study of the source: Mrk421

All data taken from the path: /fefs/aswg/LST1MAGIC/mc/DL2/ST0316A/NSB0.5/GammaTest/v01.2/dec_4822/dl2_gamma_zd_4*.h5


### Reading the MC data per runs, nd only for a determined zd

In [3]:
dir_mcs = glob.glob(path_mcs)

df_mcs = [pd.read_hdf(_dir_, key='events/parameters') for _dir_ in dir_mcs]

logger.info(f'The amount of MC files to join is {len(df_mcs)}\n')

# joining the MC data
df_mc = pd.concat(df_mcs)
df_mc.set_index(['obs_id', 'event_id'], inplace=True)
df_mc.sort_index(inplace=True)
del df_mcs

logger.info(f'The MC dl2 ({sys.getsizeof(df_mc)*1e-9:.1f}Gb) and {int(len(df_mc)/3)} events:')
display(df_mc.head(5))

The amount of MC files to join is 7

The MC dl2 (6.0Gb) and 4824891 events:


tel_id  pointing_alt  pointing_az  n_pixels  n_islands  \
obs_id event_id                                                           
106    800            1      0.816866     4.585189         7          1   
       800            3      0.816866     4.585189         9          1   
       6607           1      0.816866     4.585189        16          1   
       6607           2      0.816866     4.585189        18          1   
       6607           3      0.816866     4.585189        18          1   

                 true_energy   true_alt     true_az  true_disp  true_core_x  \
obs_id event_id                                                               
106    800          0.055094  46.863731  262.132060   0.685766   145.081787   
       800          0.055094  46.863731  262.132060   0.731627   145.081787   
       6607         0.334875  46.881773  263.283329   0.218194    27.330303   
       6607         0.334875  46.881773  263.283329   0.289865    27.330303   
       6607         0.334875  46.881773  263.283329   0.257517    27.330303   

                 true_core_y  true_impact  off_axis  magic_stereo   intensity  \
obs_id event_id                                                                 
106    800        -93.304260   174.582687  0.401368         False  187.654883   
       800        -93.304260   173.081484  0.401368         False   70.758062   
       6607        -9.935828    51.861562  0.398654          True  586.823883   
       6607        -9.935828    59.044294  0.398654          True  217.242338   
       6607        -9.935828    61.817486  0.398654          True  235.072818   

                 skewness  kurtosis         x         y         r         phi  \
obs_id event_id                                                                 
106    800       0.021378  1.639115  0.267687  0.056791  0.273645   11.977854   
       800       0.074926  1.832027  0.093766  0.094431  0.133076   45.202428   
       6607      0.480530  3.761446  0.140232  0.250836  0.287374   60.792223   
       6607     -0.067014  2.376142 -0.065226  0.108009  0.126176  121.127556   
       6607     -0.055011  2.478331 -0.009191  0.198270  0.198483   92.654209   

                   length  length_uncertainty     width  width_uncertainty  \
obs_id event_id                                                              
106    800       0.056431            0.001647  0.013701           0.001077   
       800       0.030694            0.001664  0.018051           0.001427   
       6607      0.044227            0.001517  0.028522           0.000738   
       6607      0.036384            0.001448  0.022154           0.000882   
       6607      0.029974            0.001188  0.025021           0.001012   

                       psi  intercept  deviation      slope  pixels_width_1  \
obs_id event_id                                                               
106    800       47.230791  12.452266   0.290660  -0.517868             0.0   
       800      -86.041003   8.187642   0.208907   7.789383             0.0   
       6607      19.550688  14.083917   0.617567  -0.324819             0.0   
       6607      10.532336   9.413662   0.501556   6.446793             0.0   
       6607     -41.822723   9.294282   0.427494  15.202115             0.0   

                 pixels_width_2  intensity_width_1  intensity_width_2  \
obs_id event_id                                                         
106    800                  0.0                0.0                0.0   
       800                  0.0                0.0                0.0   
       6607                 0.0                0.0                0.0   
       6607                 0.0                0.0                0.0   
       6607                 0.0                0.0                0.0   

                 multiplicity  combo_type        alt  alt_uncert          az  \
obs_id event_id                                                                
106    800                  

### Computing the mean values

In [4]:
%%time
logger.info(f'Computing the mean')
df_mean = get_dl2_mean(df_mc, weight_type=weight_type)
display(df_mean.head(5))

# --- create .h5 file --- #
aux.createdir(output_path)
logger.info(f'Creating the merged file')
df_mc.to_hdf(f'{output_path}/dl2_MC_merged_{source_name}.h5', key='events/parameters')
logger.info(f'Creating the mean file')
df_mean.to_hdf(f'{output_path}/dl2_MC_mean_{source_name}.h5', key='events/parameters')

Computing the mean


combo_type  multiplicity  true_energy   true_alt     true_az  \
obs_id event_id                                                                 
106    800                2             2     0.055094  46.863731  262.132060   
       6607               3             3     0.334875  46.881773  263.283329   
       7100               2             2     0.124289  46.944013  230.547681   
       7106               1             2     0.124289  46.944013  230.547681   
       7903               3             3     0.435743  47.063377  229.555281   

                 pointing_alt  pointing_az  reco_energy   reco_alt  \
obs_id event_id                                                      
106    800           0.816866     4.585189     0.095015  46.943664   
       6607          0.816867     4.585189     0.291133  46.979149   
       7100          0.816866     4.014345     0.134875  46.969934   
       7106          0.816866     4.014345     0.125389  46.946158   
       7903          0.816867     4.014345     0.240692  47.017525   

                    reco_az  gammaness  
obs_id event_id                         
106    800       262.187940   0.777075  
       6607      263.344145   0.981453  
       7100      230.717378   0.961351  
       7106      230.516403   0.945393  
       7903      229.339749   0.873320

Creating the merged file
Creating the mean file


CPU times: user 32.2 s, sys: 42.8 s, total: 1min 15s
Wall time: 1min 50s
